In [1]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # "0,1,2,3" # "0"

In [ ]:
import numpy as np
import h5py
import cv2
import os, sys, gc
import argparse
from glob import glob
from core.get_args import get_args
from core.produce_denoised_img import produce_denoised_img_no_crop
from core.utils import TedataLoader
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from core.watershed import watershed,watershed_per_img,watershed_original
from core.median_filter import apply_median_filter_cpu, apply_median_filter_gpu, apply_median_filter_gpu_simple
from core.median_filter import *
%load_ext autoreload
%autoreload 2

# Denoising SET1~10 image

In [2]:

# img_dict = {}
# debug = False
# # data_path = "/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung_SNU_1474x3010_aligned_ordered/" 
# for data_path in glob("/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung*SNU*"):
#     for set_num in sorted(os.listdir(data_path)):
#         if set_num[0] == '.' :
#             continue
#         img_dict[set_num] = {}
#         # if debug is True:
#         #     print(set_num)
#         set_path = os.path.join(data_path,set_num)
#         img_list = sorted(os.listdir(set_path))
#         img_list = list(filter(lambda x : ".ipynb_checkpoints" not in x,img_list))
#         f_num_list = np.array(list(map(lambda x : x.split('_')[0],img_list)))
#         f_num_list = np.unique(f_num_list)
#         for f_num in f_num_list:
#             img_dict[set_num][f_num] = None
#         # if debug is True:
#         #     print(img_list)
#         #     print(f_num_list)


#         for img_path in sorted(img_list):
#             f_num = img_path.split("_")[0]

#             img_path = os.path.join(set_path,img_path)
#             if debug is True:
#                 print(f_num," & ",img_path)
#             img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
#             # img_dict[set_num][f_num].append(img)
#             img = np.expand_dims(img,axis=0)
#             img = np.expand_dims(img,axis=0)/255.
#             if img_dict[set_num][f_num] is None:  
#                 img_dict[set_num][f_num] = img
#             else :
#                 img_dict[set_num][f_num] = np.append(img_dict[set_num][f_num],img,axis=0)
#             if debug is True:
#                 print(set_nu,f_num,img.shape)
img_dict = np.load("./intermediate_result/full_img_dict.npy",allow_pickle=True).item()

In [3]:
print(img_dict.keys())
for set_num in img_dict.keys():
    print(img_dict[set_num].keys())
    for f_num in img_dict[set_num].keys():
        print(img_dict[set_num][f_num].shape)

dict_keys(['SET01', 'SET02', 'SET03', 'SET04', 'SET05', 'SET06', 'SET07', 'SET08', 'SET09', 'SET10'])
dict_keys(['F08', 'F16', 'F32', 'F64'])
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
dict_keys(['F08', 'F16', 'F32', 'F64'])
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
dict_keys(['F08', 'F16', 'F32', 'F64'])
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
dict_keys(['F08', 'F16', 'F32', 'F64'])
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
(16, 1, 1474, 3010)
dict_keys(['F01', 'F02', 'F04', 'F08', 'F16', 'F32', 'F64'])
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
dict_keys(['F01', 'F02', 'F04', 'F08', 'F16', 'F32', 'F64'])
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
(2, 1, 1454, 2990)
dict_keys(['F01', 'F02', 'F04'

In [4]:
debug= True

model = init_median_filter_model(kernel_size=11, repeat=3).cuda()
# DENOISE IMAGE
denoised_img_dict = {}
for set_num in img_dict.keys():
    denoised_img_dict[set_num] = {}
    for f_num in img_dict[set_num].keys():
        denoised_img_dict[set_num][f_num] = None
        # img = (torch.from_numpy(img_dict[set_num][f_num]).float()/255.).cuda()
        
        # denoised_img_dict[set_num][f_num] = 
        for img in img_dict[set_num][f_num]:
            img = np.expand_dims(img, axis=0)
            img = (torch.from_numpy(img).float()).cuda()
            # print("img shape",img.shape)
            denoised_img = apply_median_filter_gpu(model,img).cpu().numpy()
            if denoised_img_dict[set_num][f_num] is None:
                denoised_img_dict[set_num][f_num] = denoised_img
            else :
                denoised_img_dict[set_num][f_num] = np.append(denoised_img_dict[set_num][f_num], denoised_img,axis=0)

        print(f"Denoising median_filter SET : {set_num}, f_num : {f_num} End")

# SAVE IMG
print(denoised_img_dict.keys())
folder_name = "./denoised_img_median_filter"
np.save(f"./intermediate_result/{folder_name}_denoised_img_dict.npy",denoised_img_dict)

for set_num in denoised_img_dict.keys():
    print(denoised_img_dict[set_num].keys())
    os.makedirs(f"{folder_name}/{set_num}", exist_ok=True)
    for f_num in denoised_img_dict[set_num].keys():
        # print(denoised_img_dict[set_num][f_num].shape)
        # for idx,img in enumerate(denoised_img_dict[set_num][f_num]):
        #     filename = f"{folder_name}/{set_num}/{f_num}_{idx:02d}.png"
        #     img = (img*255).astype('uint8')
        #     cv2.imwrite(filename, img[0])
        filename = f"{folder_name}/{set_num}/{f_num}.npy"
        np.save(filename,denoised_img_dict[set_num][f_num])
        print(f"Saving SET : {set_num}, f_num : {f_num} End")


Denoising median_filter SET : SET01, f_num : F08 End
Denoising median_filter SET : SET01, f_num : F16 End
Denoising median_filter SET : SET01, f_num : F32 End
Denoising median_filter SET : SET01, f_num : F64 End
Denoising median_filter SET : SET02, f_num : F08 End
Denoising median_filter SET : SET02, f_num : F16 End
Denoising median_filter SET : SET02, f_num : F32 End
Denoising median_filter SET : SET02, f_num : F64 End
Denoising median_filter SET : SET03, f_num : F08 End
Denoising median_filter SET : SET03, f_num : F16 End
Denoising median_filter SET : SET03, f_num : F32 End
Denoising median_filter SET : SET03, f_num : F64 End
Denoising median_filter SET : SET04, f_num : F08 End
Denoising median_filter SET : SET04, f_num : F16 End
Denoising median_filter SET : SET04, f_num : F32 End
Denoising median_filter SET : SET04, f_num : F64 End
Denoising median_filter SET : SET05, f_num : F01 End
Denoising median_filter SET : SET05, f_num : F02 End
Denoising median_filter SET : SET05, f_num : F